# Gaussian Garment finetuning and inference

This notebook shows how to finetune a ContourCraft model and material using the data generated by Gaussian Garments. This process is described in the Gaussian Garments paper as Stage 4 (Finetuning behavior).

Once you have finetuned the model, you will be able to generate trajectories with ContourCraft which then can be visualised as photorealistic garments using the Gaussian Garments inference script.

## Convert Gaussian Garments data into ContourCraft format

First, let's convert the data that is used and generated by Gaussian Garments into the format used by the model finetuning process in ContourCraft.

Before starting, make sure that
* You have stored the input data for the Gaussian Garments following its [Data Preparation instructions](https://github.com/eth-ait/Gaussian-Garments/blob/main/DataPreparation.md)
* You have run the first and second stages of Gaussian Garments so now you have folders `stage1` and `stage2` in the output directory for your subject,

In [ ]:
from utils.gaugar import GauGarConverter
from pathlib import Path
from utils.defaults import DEFAULTS

Here, you will need to set the paths to the Inputs and Outputs folders you used for Gaussian Garments. 

These paths have to be the same as the ones you have set in the defaults.py for Gaussian Garments. 
See [Gaussian Garments' Readme](https://github.com/eth-ait/Gaussian-Garments?tab=readme-ov-file#data-preparation) for details.

Then, create the `GauGarConverter` object which will convert all the data to the ContourCraft format.

In [ ]:
# Inputs forder, same as DEFAULTS.data_root in Gaussian Garments
gaugar_data_root = Path('/path/to/gaussian/garments/input/data')

# Outpus forder, same as DEFAULTS.output_root in Gaussian Garments
gaugar_output_root = Path('/path/to/gaussian/garments/output/data')

body_model_root = Path(DEFAULTS.aux_data) / 'body_models'

garment_dicts_dir = Path(DEFAULTS.aux_data) / 'garment_dicts' / 'finetune'
checkpoint_path = Path(DEFAULTS.data_root) / 'trained_models' / 'contourcraft.pth'

converter = GauGarConverter(gaugar_data_root, gaugar_output_root, body_model_root, garment_dicts_dir, checkpoint_path)

Now, set the name and the template sequence and frame for your subject. THese values should be the same as you have used for stages 1 and 2 of Gaussian Garments.

If you did not use `subject_out` parameter, set it to `None`, then the `subject` value will be used for the `subject_out` as well.

In [ ]:
# the name of the subject you used in Gaussian Garments
subject = '{subject}' 

# the name of the output subject you used in Gaussian Garments
# set to None if you want to use the same name as the input subject
subject_out = None 

# gender of the SMPL-X model you used
gender = 'female'

# the template sequence and the template frame number
# you used in stages 1 and 2 of Gaussian Garments
template_sequence = '{sequence}'
template_frame = {template_frame}

Finally, convert the training data using the `converter.convert_subject` function.

This function will perform 5 actions:
1. Convert the SMPL-X parameters stored in `{gaugar_data_root}/{subject}/*/smplx/` into an .npz pose sequences of the same format used in AMASS CMU dataset. The sequences will be stored under `{gaugar_output_root}/{subject_out}/stage4/smplx/`
2. Convert the garment mesh registrations created in stage 2 of Gaussian Garments and stored under `{gaugar_output_root}/{subject_out}/stage2/*/meshes` into .pkl mesh sequences and store them under  `{gaugar_output_root}/{subject_out}/stage4/registrations/`
3. Import the .obj template mesh of the garment stored in `{gaugar_output_root}/{subject_out}/stage1/template_uv.obj` as a garment dictionary used by ContourCraft. The garment dictionary will be stored in `{garment_dicts_dir}/{subject_out}.pkl`
4. Create .csv data lists used by the ContourCraft dataloader for finetuning, and store them under `{DEFAULTS.data_root}/aux_data/datasplits/finetuning/{subject_out}/`
5. Create a ContourCraft configuration file for the finetuning and store it as `{DEFAULTS.project_dir}/configs/finetune/{subject_out}.csv` 


In [ ]:
converter.convert_subject(subject, gender, template_sequence, template_frame, subject_out=subject_out)

### Finetune multiple garments with a single model

If you want to finetune a single model and the material parameters for multiple garments at the same time, you'll first need to convert their data individually using the code above.

Then, use the function `converter.prepare_multigarment` to create a configuration file and the datasplits for the multi-garment finetuning.

In [ ]:

# add all the subject names (subject_out) to the list
subject_list = []
subject_list.append('{subject_out_1}')
subject_list.append('{subject_out_2}')
subject_list.append('{subject_out_3}')

# set the name of the experiment, it will be used to name the configuration file and the datasplits
experiment_name = 'my_experiment'

# create the configuration file and the datasplits
converter.prepare_multigarment(subject_list, experiment_name)


## Finetune the ContourCraft model and material

Once you have converted all the data with the code above, you can start the finetuning procedure with the following command:

`python train.py config=finetune/{subject_out}`

The whole process should take around 24 hours depending on the hardware.

The finetuned checkpoints will be stored in `{DEFAULTS.data_root}/trained_models/finetuning/{subject_out}/`

## Generate sequences with the finetuned model and material

Now you can create the simulation trajectories with the finetuned bahavior. To do that, first, set the variable `checkpoint_path` below to the path of the latest finetuning checkpoint, and load the runner and material objects:

In [ ]:
from utils.validation import load_runner_and_material_from_checkpoint
from utils.arguments import load_params
from utils.defaults import DEFAULTS
from pathlib import Path
from utils.common import move2device
from utils.io import pickle_dump



# ====================================================================================================

config_name = 'finetune/base'
models_dir = Path(DEFAULTS.data_root) / 'trained_models'
checkpoint_path = models_dir / 'finetuning/{subject_out}/step_XXXXXXXXXX.pth'


# ====================================================================================================


# load the config from a .yaml file and load .py modules specified there
modules, experiment_config = load_params(config_name)

# load a Runner object and the .py module it is declared in together with the material stack
runner_module, runner, material_stack = load_runner_and_material_from_checkpoint(checkpoint_path, modules, experiment_config)


Then, choose a SMPL-X pose sequence and set `sequence_path` to its path. It can be a pose sequence from the AMASS CMU dataset, or one of the pose sequences from your data, which we created in the first part of this notebook.

After that, you can create a dataloader for this sequence and the `{subject_out}` garment...

In [ ]:
# file with the pose sequence
from utils.validation import create_postcvpr_one_sequence_dataloader

# If True, the SMPL(-X) poses are slightly modified to avoid hand-body self-penetrations. The technique is adopted from the code of SNUG 
separate_arms = True

# To test the simulation with the SMPL-X body model
CMU_path = '/path/to/AMASS/smplx/CMU'
sequence_path =  Path(CMU_path) / '01/01_01_stageii.npz'
sequence_loader = 'cmu_npz_smplx'
garment_dicts_dir = Path(DEFAULTS.aux_data) / 'garment_dicts' / 'finetune' 
garment_name = subject_out
gender = 'female'

dataloader = create_postcvpr_one_sequence_dataloader(sequence_path, garment_name, sequence_loader=sequence_loader, 
                                            obstacle_dict_file=None, gender=gender, garment_dicts_dir=garment_dicts_dir)

... and generate the simulation trajectory.

In [ ]:
sequence = next(iter(dataloader))
sequence = move2device(sequence, 'cuda:0')
trajectories_dict = runner.valid_rollout(sequence, material_stack,  bare=True, n_steps=200)

Finally, save the trajectory to a .pkl file:

In [ ]:
trajectory_file = '/your/trajectory/path.pkl'
pickle_dump(trajectories_dict, trajectory_file)

Now you can create photorealistic renders of the trajectory by using the following command in the Gaussian Garments repository:

```bash
cd /path/to/Gaussian-Garments/
python inference.py --traj_path {trajectory_file}.pkl --output_path {directory_to_store_renders}
``` 